In [5]:
import json
import pandas as pd
from feature_extractor import *
data = pd.Series.from_csv('data/505_0_3000190.csv')

SyntaxError: Missing parentheses in call to 'print' (feature_extractor.py, line 57)

In [4]:
data


time                         value
2017-01-20 08:00:00+00:00    73.66
2017-01-20 08:03:00+00:00    73.66
2017-01-20 08:06:00+00:00    73.66
2017-01-20 08:09:00+00:00    73.66
2017-01-20 08:12:00+00:00    73.66
2017-01-20 08:15:00+00:00    73.66
2017-01-20 08:18:00+00:00    73.66
2017-01-20 08:21:00+00:00    73.66
2017-01-20 08:24:00+00:00    73.66
2017-01-20 08:27:00+00:00    73.66
2017-01-20 08:30:00+00:00    73.66
2017-01-20 08:33:00+00:00    73.66
2017-01-20 08:36:00+00:00    73.66
2017-01-20 08:39:00+00:00    73.66
2017-01-20 08:42:00+00:00    73.67
2017-01-20 08:45:00+00:00    73.67
2017-01-20 08:48:00+00:00    73.66
2017-01-20 08:51:00+00:00    73.66
2017-01-20 08:54:00+00:00    73.66
2017-01-20 08:57:00+00:00    73.67
2017-01-20 09:00:00+00:00    73.67
2017-01-20 09:03:00+00:00    73.66
2017-01-20 09:06:00+00:00    73.67
2017-01-20 09:09:00+00:00    73.66
2017-01-20 09:12:00+00:00    73.66
2017-01-20 09:15:00+00:00    73.67
2017-01-20 09:18:00+00:00    73.66
2017-01-20 09:21:00+

In [38]:
#essential libraries                                                            
import numpy as np                                                              
import pandas as pd      
import arrow
import json
                                                                                
from bd3client.CentralService import CentralService                             
from bd3client.Sensor import Sensor                                             
from bd3client.Timeseries import Timeseries                                     
                                                                                
from building_depot import DataService, BDError                                 
from datetime import datetime
                                                                                
import logging                                                                     
                                                                                
# Basic configuration                                                           
begin_time = arrow.get(datetime(2017,2,12,13), 'US/Pacific').datetime
end_time = arrow.get(datetime(2017,2,12,14), 'US/Pacific').datetime                                      
building_name_list = ['AP_M', 'EBU3B', 'SME', 'Music']                          
basedir = "data"                                                                
header = ['value']                                                                 
index_label="time" 

# BD2-1 Initialization                                                          
with open("config/bd2-1config.json", "r") as fp:                                
    hostname = json.load(fp)['hostname']                                        
with open("config/bd2-1secrets.json", "r") as fp:                               
    secrets = json.load(fp)                                           
    username = secrets['username']                                              
    apikey = secrets['apikey']                                                  
bd2_1ds = DataService(hostname, apikey, username) 

srcid = '506_0_3000485'


# Get UUID                                                                  
uuid = bd2_1ds.list_sensors({'source_identifier':srcid})['sensors'][0]['uuid']

# Get data                                                                  

raw_data = bd2_1ds.get_timeseries_datapoints(uuid, 'PresentValue', begin_time, end_time)

raw_data['timeseries'][-1]

{'2017-02-12T21:59:31+00:00': 78.92}

In [102]:
#essential libraries
import json
import logging

import numpy as np
import pandas as pd
import arrow

from bd3client.CentralService import CentralService
from bd3client.Sensor import Sensor
from bd3client.Timeseries import Timeseries
from building_depot import DataService, BDError

PST = 'US/Pacific'

# Basic configuration
begin_time = arrow.get(datetime(2017,1,20), PST).datetime
end_time = arrow.get(datetime(2017,2,8), PST).datetime
#building_name_list = ['AP_M', 'EBU3B', 'SME', 'Music']
building_name_list = ['EBU3B']
basedir = "data"
header = ['value']
index_label="time"

# Logger configuration
logger = logging.getLogger("data_collection_log")
logger.setLevel(logging.INFO)
log_handler = logging.FileHandler('log/data_collection.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
logger.addHandler(log_handler)

# BD2-1 Initialization
with open("config/bd2-1config.json", "r") as fp:
    hostname = json.load(fp)['hostname']
with open("config/bd2-1secrets.json", "r") as fp:
    secrets = json.load(fp)
    username = secrets['username']
    apikey = secrets['apikey']
bd2_1ds = DataService(hostname, apikey, username)

# BD3 Initialization
with open("config/bd3config.json", "r") as fp:
    hostname = json.load(fp)['hostname']
with open("config/bd3secrets.json", "r") as fp:
    secrets = json.load(fp)
    cid = secrets['cid']
    csecret = secrets['csecret']
bd3cs = CentralService(hostname, cid, csecret)
bd3sensor_api = Sensor(bd3cs)
bd3ts_api = Timeseries(bd3cs)


#Load basic metadata
with open("metadata/bacnet_devices.json", "r") as fp:
    bacnet_devices = json.load(fp)
with open("metadata/building_info.json", "r") as fp:
    building_dict = json.load(fp)

def resample_data(raw_data, begin_time, end_time, sample_method):
    raw_data = raw_data[begin_time:end_time]
    if not begin_time in raw_data.index:
        raw_data[arrow.get(begin_time).to('UTC').datetime] = raw_data.head(1)[0]
    if not end_time in raw_data.index:
        raw_data[arrow.get(end_time).to('UTC').datetime] = raw_data.tail(1)[0]
    raw_data = raw_data.sort_index()
    if sample_method== 'nextval':
        proc_data = raw_data.resample('3Min', how='pad')
    else:
        logger.error("sample method not defined well: {0}".format(sample_method))
        assert(False)
    return proc_data


def get_data_bd2(begin_time, end_time, srcid, bd2ds):
    # Get UUID
    """
    try:
        uuid = bd2ds.list_sensors({'source_identifier':srcid})\
                ['sensors'][0]['uuid']
    except:
        logger.error("UUID not found ({0})".format(srcid))
        return None
    """
    uuid = bd2ds.list_sensors({'source_identifier':srcid})\
            ['sensors'][0]['uuid']

    # Get data
    """
    try:
        raw_data = bd2ds.get_timeseries_datapoints(uuid, 'PresentValue', \
                begin_time, end_time)
    except:
        logger.error("Data not found ({0})".format(srcid))
        return None
    """
    raw_data = bd2ds.get_timeseries_datapoints(uuid, 'PresentValue', \
            begin_time, end_time)
    time_list = list()
    value_list = list()
    for row in raw_data['timeseries']:
        for t, v in row.items():
            time_str = t
            value = v
            break
        from dateutil import tz
        time_list.append(arrow.get(time_str).to('UTC').datetime)
        value_list.append(value)
    return pd.Series(index=time_list, data=value_list)
    

def get_data_bd3(begin_time, end_time, srcid):
    # Get UUID
    try:
        (nae, data_type, instance_num)  = srcid.split("_")
        tag_val_dict = {
                "NAENum": nae,
                "BACNet_DataType": data_type,
                "BANet_InstanceNumber": instance_num
                }
        uuid = bd3sensor_api.search(tag_val_dict)['result'][0]['name']
    except:
        logger.error("UUID NOT FOUND ({0}, {1})".format(building_name, srcid))
        return None
    # Get data
    try:
        ts = bd3ts_api.getTimeseriesDataPoints(uuid, begin_time, end_time)
    except:
        logger.error("Data not found ({0})".format(srcid))
    time_list = [arrow.get(row[0]).to('UTC').datetime for row in ts['data']['series'][0]\
            ['values']]
    value_list = [row[2] for row in ts['data']['series'][0]['values']]
    return pd.Series(index=time_list, data=value_list)
    #.to_csv(filename, header=header, index_label=index_label)

def get_data(building_name, begin_time, end_time, srcid):
    if building_name.lower() == "ebu3b":
        return get_data_bd2(begin_time, end_time, srcid, bd2_1ds)
    else:
        return get_data_bd3(begin_time, end_time, srcid)

"""
if __name__ == "__main__":
    cnt = 0
    # Store all data
    for building_name in building_name_list:
        cnt += 1
        if cnt%500==0:
            logger.info("Data download success ({0}, {1})"\
                    .format(building_name, cnt))
        with open("metadata/%s_sentence_dict.json"%building_name.lower(), "r") \
                as fp:
            sentence_dict = json.load(fp)
        srcid_list = sentence_dict.keys()
        for srcid in srcid_list:
            ts_series = get_data(building_name, begin_time, end_time, srcid)
            if isinstance(ts_series, pd.Series):
                ts_series = resample_data(ts_series, begin_time, end_time, \
                        "nextval")
                filename = basedir + '/' + srcid + '.csv'
                ts_series.to_csv(filename, \
                        header=header, \
                        index_label=index_label)
            else:
                logger.error("Data not found ({0})".format(srcid))
"""

'\nif __name__ == "__main__":\n    cnt = 0\n    # Store all data\n    for building_name in building_name_list:\n        cnt += 1\n        if cnt%500==0:\n            logger.info("Data download success ({0}, {1})"                    .format(building_name, cnt))\n        with open("metadata/%s_sentence_dict.json"%building_name.lower(), "r")                 as fp:\n            sentence_dict = json.load(fp)\n        srcid_list = sentence_dict.keys()\n        for srcid in srcid_list:\n            ts_series = get_data(building_name, begin_time, end_time, srcid)\n            if isinstance(ts_series, pd.Series):\n                ts_series = resample_data(ts_series, begin_time, end_time,                         "nextval")\n                filename = basedir + \'/\' + srcid + \'.csv\'\n                ts_series.to_csv(filename,                         header=header,                         index_label=index_label)\n            else:\n                logger.error("Data not found ({0})".format(src

In [103]:
srcid = '506_0_3000485'
building_name = 'ebu3b'
ts = get_data(building_name, begin_time, end_time, srcid)

In [96]:
ts.index.ravel()[0]

numpy.datetime64('2017-01-20T08:02:19.000000000')

In [97]:
begin_time

datetime.datetime(2017, 1, 20, 0, 0, tzinfo=tzfile('/usr/share/zoneinfo/US/Pacific'))

In [104]:
isinstance(ts, pd.Series)
rd = resample_data(ts, begin_time, end_time, 'nextval')


/home/jbkoh/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:68: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).pad()


In [105]:
rd

2017-01-20 08:00:00+00:00    76.20
2017-01-20 08:02:00+00:00    76.20
2017-01-20 08:04:00+00:00    76.20
2017-01-20 08:06:00+00:00    76.20
2017-01-20 08:08:00+00:00    76.20
2017-01-20 08:10:00+00:00    76.20
2017-01-20 08:12:00+00:00    76.20
2017-01-20 08:14:00+00:00    76.20
2017-01-20 08:16:00+00:00    76.71
2017-01-20 08:18:00+00:00    76.71
2017-01-20 08:20:00+00:00    76.71
2017-01-20 08:22:00+00:00    76.71
2017-01-20 08:24:00+00:00    76.71
2017-01-20 08:26:00+00:00    76.71
2017-01-20 08:28:00+00:00    76.71
2017-01-20 08:30:00+00:00    76.71
2017-01-20 08:32:00+00:00    76.21
2017-01-20 08:34:00+00:00    76.21
2017-01-20 08:36:00+00:00    76.21
2017-01-20 08:38:00+00:00    76.21
2017-01-20 08:40:00+00:00    76.21
2017-01-20 08:42:00+00:00    76.21
2017-01-20 08:44:00+00:00    76.21
2017-01-20 08:46:00+00:00    76.21
2017-01-20 08:48:00+00:00    75.70
2017-01-20 08:50:00+00:00    75.70
2017-01-20 08:52:00+00:00    75.70
2017-01-20 08:54:00+00:00    75.70
2017-01-20 08:56:00+

In [98]:
ts.index.ravel()[0]

numpy.datetime64('2017-01-20T08:02:19.000000000')

In [68]:
from datetime import tzinfo
begin_time.astimezone(tzinfo.tzname('UTC'))

TypeError: descriptor 'tzname' requires a 'datetime.tzinfo' object but received a 'str'

In [99]:
a = arrow.get(begin_time).to('UTC').datetime

In [86]:
b = ts.index[0]

In [87]:
b

Timestamp('2017-01-20 08:02:19+0000', tz='tzoffset(None, 0)')

In [101]:
ts

2017-01-20 08:02:19+00:00    76.20
2017-01-20 08:06:31+00:00    76.20
2017-01-20 08:10:45+00:00    76.20
2017-01-20 08:14:45+00:00    76.71
2017-01-20 08:18:43+00:00    76.71
2017-01-20 08:22:43+00:00    76.71
2017-01-20 08:26:54+00:00    76.71
2017-01-20 08:30:50+00:00    76.21
2017-01-20 08:34:51+00:00    76.21
2017-01-20 08:38:53+00:00    76.21
2017-01-20 08:42:43+00:00    76.21
2017-01-20 08:46:48+00:00    75.70
2017-01-20 08:50:43+00:00    75.70
2017-01-20 08:54:36+00:00    75.70
2017-01-20 08:58:33+00:00    75.70
2017-01-20 09:02:24+00:00    75.70
2017-01-20 09:06:26+00:00    76.23
2017-01-20 09:10:21+00:00    76.23
2017-01-20 09:14:19+00:00    76.76
2017-01-20 09:18:14+00:00    76.76
2017-01-20 09:22:10+00:00    76.25
2017-01-20 09:26:13+00:00    76.25
2017-01-20 09:30:07+00:00    76.25
2017-01-20 09:34:06+00:00    75.74
2017-01-20 09:38:00+00:00    75.74
2017-01-20 09:42:00+00:00    75.74
2017-01-20 09:46:03+00:00    75.74
2017-01-20 09:50:09+00:00    75.74
2017-01-20 09:54:15+

In [45]:
ts.resample('3T', how='pad')

/home/jbkoh/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).pad()
  if __name__ == '__main__':


2017-01-20 08:00:00+00:00      NaN
2017-01-20 08:03:00+00:00    76.20
2017-01-20 08:06:00+00:00    76.20
2017-01-20 08:09:00+00:00    76.20
2017-01-20 08:12:00+00:00    76.20
2017-01-20 08:15:00+00:00    76.71
2017-01-20 08:18:00+00:00    76.71
2017-01-20 08:21:00+00:00    76.71
2017-01-20 08:24:00+00:00    76.71
2017-01-20 08:27:00+00:00    76.71
2017-01-20 08:30:00+00:00    76.71
2017-01-20 08:33:00+00:00    76.21
2017-01-20 08:36:00+00:00    76.21
2017-01-20 08:39:00+00:00    76.21
2017-01-20 08:42:00+00:00    76.21
2017-01-20 08:45:00+00:00    76.21
2017-01-20 08:48:00+00:00    75.70
2017-01-20 08:51:00+00:00    75.70
2017-01-20 08:54:00+00:00    75.70
2017-01-20 08:57:00+00:00    75.70
2017-01-20 09:00:00+00:00    75.70
2017-01-20 09:03:00+00:00    75.70
2017-01-20 09:06:00+00:00    75.70
2017-01-20 09:09:00+00:00    76.23
2017-01-20 09:12:00+00:00    76.23
2017-01-20 09:15:00+00:00    76.76
2017-01-20 09:18:00+00:00    76.76
2017-01-20 09:21:00+00:00    76.76
2017-01-20 09:24:00+